In [103]:
import time
import datetime
import os
import yaml
import math
import sys
import gurobipy
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

print(f"python version: {sys.version}")
print(f"gurobipy version: {gurobipy.gurobi.version()}")


python version: 3.9.6 (default, Sep 26 2022, 11:37:49) 
[Clang 14.0.0 (clang-1400.0.29.202)]
gurobipy version: (11, 0, 0)


In [104]:
# 添加相對路徑到 sys.path
sys.path.append('../')  
from path_config import INSTANCES_DIR
def load_specific_yaml(filename):
    """
    加載指定的 YAML 檔案。
    
    Parameters:
    filename (str): 在 instances 資料夾中的 YAML 檔案名。
    
    Returns:
    dict: YAML 檔案內容。
    """
    file_path = os.path.join(INSTANCES_DIR, filename)
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

# Load experiments yaml
config = load_specific_yaml('instance_F_4.yaml')
config['lambda_for_G'] = 3e-6
config['M'] = 1000000
config

{'A_opponent_bar': [29759, 33774, 91379, 58452, 57268, 122319],
 'B': [[2, 2, 3],
  [1, 1, 3],
  [4, 2, 1],
  [3, 1, 2],
  [2, 4, 1],
  [4, 1, 2],
  [2, 3, 3],
  [1, 2, 2],
  [2, 4, 3],
  [4, 4, 1],
  [4, 1, 1],
  [2, 2, 2]],
 'C': [1, 3, 1, 2, 1, 1, 2, 2, 3, 3, 2, 3],
 'D': [[68.62,
   89.27,
   23.35,
   30.15,
   82.88,
   80.92,
   63.2,
   90.62,
   62.24,
   35.36,
   57.87,
   49.77],
  [39.62,
   39.92,
   53.31,
   83.49,
   72.69,
   81.49,
   15.0,
   20.81,
   17.0,
   41.0,
   38.91,
   82.33],
  [65.31,
   85.07,
   19.72,
   36.12,
   82.81,
   82.1,
   57.01,
   82.87,
   54.42,
   27.31,
   54.56,
   53.6],
  [25.71,
   47.68,
   29.41,
   37.05,
   34.83,
   36.06,
   40.22,
   76.03,
   55.66,
   45.89,
   18.03,
   27.66],
  [18.87,
   39.85,
   38.21,
   46.04,
   26.42,
   30.15,
   39.56,
   74.71,
   57.56,
   52.2,
   15.23,
   33.11]],
 'D_comp': [[71.85, 74.46, 14.14, 66.48, 97.08, 61.07],
  [28.02, 2.0, 62.39, 67.42, 43.57, 61.33],
  [67.19, 67.27, 5.66, 66.

In [105]:
I = set(range(0, config['i_amount']))  
J = set(range(0, config['j_amount']))  
K = set(range(0, config['k_amount']))  
L = set(range(0, config['l_amount']))  
'''
Decision Variables
===============================
y : Whether location j build facility or not, Dim: (j) , BINARY
x : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS
aX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS
===============================

Other Variables
u : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS
P : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS
TA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS
===============================
'''

'\nDecision Variables\n===============================\ny : Whether location j build facility or not, Dim: (j) , BINARY\nx : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS\naX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS\n===============================\n\nOther Variables\nu : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS\nP : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS\nTA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS\n===============================\n'

In [106]:
# # Model initial (G, no reformulate)
# model = Model("Competitive Facility Location")
# model.setParam('NumericFocus', 3)
# y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
# x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
# aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

# u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="utility(u)", lb=0) # utility var(3.5)
# P = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="P, utility vs TA ratio", lb=0.0, ub=1.0) # Probability of i choosing j
# TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000000)
# TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="negative TAi*lambda(e's power)", lb= -100000, ub= -0.000001) #-lambda*TAi
# G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars", lb=0.0, ub=1) #e^-(lambda*TAi), ub=0.5

# model.update()

# model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
# model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
# model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
# model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

# model.addConstrs((u[i, j] == ((quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j]) / (config['D'][i][j]**2)) for i in I for j in J), "(3.5) utility for facility j to customer i")
# model.addConstrs((u[i, j] == P[i, j] * TA[i] for i in I for j in J), "(3.8) calculate P by uij/TAi, 分母要是常數所以用乘的")
# ## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
# model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + (quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) for i in I), "(3.7) calculate total TA by u and A_bar")

# ## G的計算
# # def G(tai):
# #     return 1 - np.exp(-config['lambda_for_G'] * tai)
# model.addConstrs((TAi_lambda[i] == (-config['lambda_for_G'] * TA[i]) for i in I), "negative tai*lambda constr")
# # Gurobi無法直接在目標式有exponential
# for i in I:
#     model.addGenConstrExpA(TAi_lambda[i], G_exp_vars[i], math.e, name=f"G_exp_constr_{i}")



# # Objective function: Maximize profit by attracting customers - costs
# objective = ((quicksum(config['H'][i] * (1 - G_exp_vars[i]) * (quicksum(P[i, j] for j in J)) for i in I)) \
#           - (quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J)))
# model.setObjective(objective, GRB.MAXIMIZE)


# # Solve and Output
# model.optimize()
# if model.status == GRB.OPTIMAL:
#     print("Optimal solution found.")
# else:
#     print("No optimal solution found.")
#     model.computeIIS()
#     # 将IIS输出到文件
#     # model.write("model_iis.mps")
#     print("IIS written to file 'model_iis.mps'")



In [107]:
# Model  (G, reformulate)
model = Model("Competitive Facility Location")
model.setParam('NumericFocus', 3)
y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="utility(u)", lb=0) # utility var(3.5)
P = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="P, opponent vs TA ratio", lb=0.0, ub=1.0) # 改這裡
TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000000)
TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="negative TAi*lambda(e's power)", lb= -100000, ub= -0.000001) #-lambda*TAi
G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars", lb=0.0, ub=1) #e^-(lambda*TAi), ub=0.5

model.update()

model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

model.addConstrs((u[i, j] == ((quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j]) / (config['D'][i][j]**2)) for i in I for j in J), "(3.5) utility for facility j to customer i")
## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + (quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) for i in I), "(3.7) calculate total TA by u and A_bar")

model.addConstrs(((quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) \
                  == P[i, j] * TA[i] for i in I for j in J), "A_opponent對比TAI, 分母要是常數所以用乘的")

## G的計算
# def G(tai):
#     return 1 - np.exp(-config['lambda_for_G'] * tai)
model.addConstrs((TAi_lambda[i] == (-config['lambda_for_G'] * TA[i]) for i in I), "negative tai*lambda constr")
# Gurobi無法直接在目標式有exponential
for i in I:
    model.addGenConstrExpA(TAi_lambda[i], G_exp_vars[i], math.e, name=f"G_exp_constr_{i}")



# Objective function: Maximize profit by attracting customers - costs
objective = (quicksum(config['H'][i] * (1 - G_exp_vars[i]) * (1 - quicksum(P[i, j] for j in J)) for i in I)) \
          - (quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J))
model.setObjective(objective, GRB.MAXIMIZE)


# Solve and Output
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
else:
    print("No optimal solution found.")
    model.computeIIS()
    # 将IIS输出到文件
    # model.write("model_iis.mps")
    print("IIS written to file 'model_iis.mps'")



Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.5.0 21F79)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 133 rows, 195 columns and 543 nonzeros
Model fingerprint: 0xda14f052
Model has 60 quadratic objective terms
Model has 60 quadratic constraints
Model has 5 general constraints
Variable types: 183 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [3e-06, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 4e+01]
  QObjective range [4e+01, 8e+01]
  Bounds range     [1e-06, 1e+05]
  RHS range        [5e+00, 9e+03]
  QRHS range       [5e+02, 9e+03]
Presolve added 0 rows and 87 columns
Presolve removed 39 rows and 0 columns
Presolve time: 0.00s
Presolved: 455 rows, 344 columns, 1329 nonzeros
Presolved model has 5 SOS constraint(s)
Presolved model has 120 bilinear constraint(s)


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   83.11322    0  122  -12.98819   83.11322   740%     -    0s
H    0     0                      76.2458550   82.24833  7.87%     -    0s
     0     2   82.24833    0  122   76.24586   82.24833  7.87%     -    0s
H   29    15                      76.8661189   79.81452  3.84%  14.0    0s

Explored 325 nodes (6978 simplex iterations) in 0.33 seconds (0.28 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 76.8661 76.2459 -12.9882 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.686610959856e+01, best bound 7.686611893537e+01, gap 0.0000%
Optimal solution found.


In [108]:
for var in model.getVars():
    print(f"{var.varName} = {var.x}")
print("")

y[0] = -0.0
y[1] = -0.0
y[2] = 1.0
y[3] = -0.0
y[4] = -0.0
y[5] = -0.0
y[6] = 1.0
y[7] = -0.0
y[8] = -0.0
y[9] = -0.0
y[10] = 1.0
y[11] = -0.0
x[0,0] = 0.0
x[0,1] = 0.0
x[0,2] = 0.0
x[1,0] = 0.0
x[1,1] = 0.0
x[1,2] = 0.0
x[2,0] = 2.911391563072806
x[2,1] = 2.0
x[2,2] = 0.0
x[3,0] = 0.0
x[3,1] = 0.0
x[3,2] = 0.0
x[4,0] = 0.0
x[4,1] = 0.0
x[4,2] = 0.0
x[5,0] = 0.0
x[5,1] = 0.0
x[5,2] = 0.0
x[6,0] = 0.0
x[6,1] = 0.0
x[6,2] = 2.090749699180148
x[7,0] = 0.0
x[7,1] = 0.0
x[7,2] = 0.0
x[8,0] = 0.0
x[8,1] = 0.0
x[8,2] = 0.0
x[9,0] = 0.0
x[9,1] = 0.0
x[9,2] = 0.0
x[10,0] = 0.0
x[10,1] = 1.0
x[10,2] = 4.0
x[11,0] = 0.0
x[11,1] = 0.0
x[11,2] = 0.0
aX[0] = 0.0
aX[1] = 0.0
aX[2] = 0.0
aX[3] = 0.0
aX[4] = 0.0
aX[5] = 0.0
aX[6] = 0.0
aX[7] = 0.0
aX[8] = 0.0
aX[9] = 0.0
aX[10] = 0.0
aX[11] = 0.0
utility(u)[0,0] = 0.0
utility(u)[0,1] = 0.0
utility(u)[0,2] = 285849.7088122197
utility(u)[0,3] = 0.0
utility(u)[0,4] = 0.0
utility(u)[0,5] = 0.0
utility(u)[0,6] = 25850.81687120057
utility(u)[0,7] = 0.0
utili

# References

## G calculation substitude exponential

https://www.gurobi.com/documentation/10.0/refman/py_model_agc_exp.html

https://support.gurobi.com/hc/en-us/community/posts/360077178491-Build-an-objective-function-with-Log-and-Exponential-inside-

https://support.gurobi.com/hc/en-us/community/posts/10481250723217-How-to-add-log-and-exponential-term-in-objective-function
